In [ ]:
%pip install comet_ml #libreria a instalar

In [ ]:
import comet_ml
comet_ml.init(project_name="RNA_Densa_M1") #nombre a dar al experimento

In [ ]:
# Imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist #base de datos
from tensorflow.keras.models import Sequential # checar cada uno de los que faltan
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras import regularizers
from keras.callbacks import ModelCheckpoint # sirve para dar respuesta del proceso de entrenamiento de la red

In [ ]:
experiment = comet_ml.Experiment(  # Para activar las Graficaciones en comet
    auto_histogram_weight_logging=True,
    auto_histogram_gradient_logging=True,
    auto_histogram_activation_logging=True,
    log_code=True, # para subir el codigo a comet para no perder el progreso
)

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/nicglowss/rna-densa-m1/bffd8510fa3f457b8073fd7d6d95bc6a
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET INFO:     notebook            : 2
COMET INFO:     os packages         : 1
COMET INFO:     source_code         : 1
COMET INFO: 
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Reposit

In [ ]:
# pre-procesamiento
dataset=mnist.load_data()# en el dataset se guardan todas las imagenes del mnist con sus etiquetas
(x_train, y_train), (x_test, y_test) = dataset

# se van a guardar en el forato de binario
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# =numero de pixeles normalizando el no. de pixeles
x_train /= 255  # x_trainv = x_trainv/255
x_test /= 255

#clasificacion de clases
num_classes=10
y_trainc = keras.utils.to_categorical(y_train, num_classes) #analiza los datos y los acomoda en las distintas clases
y_testc = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
# Parametros
parameters = {
    "batch_size": 20, #aumentamos el tamaño de 10 a 20/50/100
    "epochs": 30,
    "optimizer": "SGD",
    "loss": "categorical_crossentropy",
}

experiment.log_parameters(parameters) # diccionario subido a comet

In [ ]:
#Arquitetura de la red
model = Sequential()
# aqui empezamos
model.add(Input(shape=(28,28))) #Flaten no tiene la opcion input_shape por lo tanto se tiene que agregar esta capa
                                #28x28 es el tamaño de la imagen
model.add(Flatten()) #Otra forma de aplanar las imagenes, las forma en una linea
model.add(Dense(784, activation='sigmoid')) # esta es la capa densa de no neuronas de forma vertical (en este caso agarramos el total de pixeles)
                                            # su funcion de activacion es la sigmoide (respuesta de la neurona de 0 a 1)
                                            # Estas dos son las principales

model.add(Dense(10, activation='softmax'))  # son las de salida literal porque estan al final

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_4 (Flatten)         (None, 784)               0         
                                                                 
 dense_8 (Dense)             (None, 784)               615440    
                                                                 
 dense_9 (Dense)             (None, 10)                7850      
                                                                 
Total params: 623290 (2.38 MB)
Trainable params: 623290 (2.38 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# specify the path where you want to save the model
filepath = "mejor-modelo1.1.hdf5"

# initialize the ModelCheckpoint callback
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [ ]:
# Inicia el entrenamiento de la red
model.compile(loss=parameters['loss'],optimizer=SGD(learning_rate=0.1,ema_momentum=0.9),metrics=['accuracy'])
model.fit(x_train, y_trainc,
                    batch_size=parameters['batch_size'],
                    epochs=parameters["epochs"],
                    verbose=1,
                    validation_data=(x_test, y_testc),
                    callbacks=[checkpoint])

Epoch 1/30
2997/3000 [============================>.] - ETA: 0s - loss: 0.5058 - accuracy: 0.8500
Epoch 1: val_loss improved from inf to 0.29369, saving model to mejor-modelo1.1.hdf5
3000/3000 [==============================] - 16s 5ms/step - loss: 0.5055 - accuracy: 0.8501 - val_loss: 0.2937 - val_accuracy: 0.9153
Epoch 2/30
2999/3000 [============================>.] - ETA: 0s - loss: 0.3054 - accuracy: 0.9106
Epoch 2: val_loss improved from 0.29369 to 0.26266, saving model to mejor-modelo1.1.hdf5
3000/3000 [==============================] - 18s 6ms/step - loss: 0.3053 - accuracy: 0.9106 - val_loss: 0.2627 - val_accuracy: 0.9237
Epoch 3/30
2995/3000 [============================>.] - ETA: 0s - loss: 0.2645 - accuracy: 0.9234
Epoch 3: val_loss improved from 0.26266 to 0.23846, saving model to mejor-modelo1.1.hdf5
3000/3000 [==============================] - 16s 5ms/step - loss: 0.2645 - accuracy: 0.9234 - val_loss: 0.2385 - val_accuracy: 0.9311
Epoch 4/30
2998/3000 [===================

In [ ]:
experiment.log_model("MNIST1", "mejor-modelo1.1.hdf5")

{'web': 'https://www.comet.com/api/asset/download?assetId=ffe0099cf22e417b94b14f9e9aaecc0b&experimentKey=396d700a7d05406a8309a6e9d14a7cbd',
 'api': 'https://www.comet.com/api/rest/v2/experiment/asset/get-asset?assetId=ffe0099cf22e417b94b14f9e9aaecc0b&experimentKey=396d700a7d05406a8309a6e9d14a7cbd',
 'assetId': 'ffe0099cf22e417b94b14f9e9aaecc0b'}

In [ ]:
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/nicglowss/rna-densa-m1/396d700a7d05406a8309a6e9d14a7cbd
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [30]                  : (0.8500666618347168, 0.9926833510398865)
COMET INFO:     batch_accuracy [9000]          : (0.09090909361839294, 1.0)
COMET INFO:     batch_loss [9000]              : (0.003752810414880514, 4.675194263458252)
COMET INFO:     epoch_duration [30]            : (15.089019902000473, 21.038901111000087)
COMET INFO:     loss [30]                      : (0.02912626788020134, 0.5054847598075867)
COMET INFO:     val_accuracy [30]              : (0.9153000116348267, 0.9785000085830688)
COMET INFO